# Introduction
This notebook shows how to train neural cross-lingual summarization (cls) presented in paper [NCLS: Neural Cross-Lingual Summarization]( https://arxiv.org/abs/1909.00156) with Th2En dataset. Their original codes are available [here](https://github.com/ZNLP/NCLS-Corpora). In this notebook, we focus only on CLS+MT (jointly train cross-lingual summarization with machine translation). <br><br>
**Neural Cross-Lingual Summarization (NCLS)** is an end-to-end cross-lingual summarization framework. The architecture of the model is based entirely on Transformer with multi-task learning: CLS+MT, and CLS+MS (cross-lingual summarization + monolingual summarization).  

---

In CLS+MT, <br>
`task1` refers to machine translation task. <br>
`task2` refers to cross-lingual summarization task.

# Install requirements

In [1]:
!pip install -q torch==1.5.1 torchvision==0.6.1
!pip install -q rouge
!pip install -q bert_score

import pandas as pd
from tqdm.notebook import tqdm
import rouge
from bert_score import score

     |████████████████████████████████| 753.2MB 22kB/s 
     |████████████████████████████████| 6.6MB 9.8MB/s 
     |████████████████████████████████| 61kB 3.0MB/s 
     |████████████████████████████████| 1.3MB 8.9MB/s 
     |████████████████████████████████| 2.9MB 46.9MB/s 
     |████████████████████████████████| 890kB 31.3MB/s 
     |████████████████████████████████| 1.1MB 42.6MB/s 


In [2]:
! git clone https://github.com/nakhunchumpolsathien/NCLS-Corpora
%cd '/content/NCLS-Corpora/code/beaver-2task+'

Cloning into 'NCLS-Corpora'...
remote: Enumerating objects: 117, done.
remote: Counting objects: 100% (117/117), done.
remote: Compressing objects: 100% (76/76), done.
remote: Total 117 (delta 38), reused 107 (delta 35), pack-reused 0
Receiving objects: 100% (117/117), 34.23 KiB | 6.84 MiB/s, done.
Resolving deltas: 100% (38/38), done.
/content/NCLS-Corpora/code/beaver-2task+


# Jointly train cross-lingual summarization + machine translation

## Build vocabulary files

In [3]:
# source language vocab
!python '/content/NCLS-Corpora/code/beaver-2task+/tools/build_vocab.py' 50000 < '/content/drive/My Drive/Projects/Model_Checkpoints/cross-lingual-projects/NCLS/beaver-2task+/th2en/trained-on-full-dataset/train.CLS.source.language1' > '/content/NCLS-Corpora/code/beaver-2task+/sample_data/vocab.source'

total words: 107850569
words in vocab: 107378135
vocab coverage: 99.56%
total unique words: 236696


In [4]:
# target language vocab
!python '/content/NCLS-Corpora/code/beaver-2task+/tools/build_vocab.py' 80000 < '/content/drive/My Drive/Projects/Model_Checkpoints/cross-lingual-projects/NCLS/beaver-2task+/th2en/trained-on-full-dataset/train.CLS.target.language2' > '/content/NCLS-Corpora/code/beaver-2task+/sample_data/vocab.target'

total words: 3172964
words in vocab: 3153813
vocab coverage: 99.40%
total unique words: 99151


## Train CLS+MT

In [5]:
!CUDA_VISIBLE_DEVICES=0, python train.py -config run_config/train-example.json

2020-11-02 06:31:38,797 - 
     batch_size | 2048
      beam_size | 4
         config | run_config/train-example.json
        dropout | 0.1
        ff_size | 2048
     grad_accum | 2
          heads | 8
    hidden_size | 512
label_smoothing | 0.1
         layers | 6
 length_penalty | 0.6
             lr | 1.0
     max_length | 200
    max_to_keep | 50
     min_length | 1
     model_path | /content/NCLS-Corpora/code/beaver-2task+/model_checkpoint
           mono | False
   report_every | 100
     save_every | 5000
share_cn_embedding | True
             tf | True
          train | ['/content/drive/My Drive/Projects/Model_Checkpoints/cross-lingual-projects/NCLS/beaver-2task+/th2en/trained-on-full-dataset/train.MT.source.language1', '/content/drive/My Drive/Projects/Model_Checkpoints/cross-lingual-projects/NCLS/beaver-2task+/th2en/trained-on-full-dataset/train.MT.target.language2', '/content/drive/My Drive/Projects/Model_Checkpoints/cross-lingual-projects/NCLS/beaver-2task+/th2en/trained-o

# Evaluation 

In [9]:
!CUDA_VISIBLE_DEVICES=0, python translate.py -config "/content/NCLS-Corpora/code/beaver-2task+/run_config/decode-example.json"

2020-11-02 06:36:37,059 - Build dataset...
2020-11-02 06:36:37,151 - Load checkpoint from /content/drive/My Drive/Projects/Model_Checkpoints/cross-lingual-projects/NCLS/beaver-2task+/th2en/trained-on-full-dataset/checkpoint-step-990000.
2020-11-02 06:36:38,692 - Build model...
2020-11-02 06:36:42,706 - Start translation...
2020-11-02 06:36:43,168 - Task 1:       1/      1
2020-11-02 06:36:49,940 - Task 2:       1/      1
2020-11-02 06:36:49,941 - Translation finished. 


# ROUGE Score & BertScore

## Output

In [10]:
print('Article: ')
with open("/content/NCLS-Corpora/code/beaver-2task+/sample_data/cls-test-set.txt") as f:
    for line in f:
        print(line)

Article: 
เว็บไซต์ nikkei asian review รายงาน อ้าง งานวิจัย โดย บริษัท cigna บริษัท บริการ ด้าน สุขภาพ ของ สหรัฐฯ ที่ ใช้การ สอบถาม กลุ่มตัวอย่าง ใน ไทย สิงคโปร์ ฮ่องกง จีน แผ่นดินใหญ่ ไป จนถึง สเปน สหรัฐอาหรับเอมิเรตส์ อังกฤษ และ สหรัฐอเมริกา งานวิจัย พบ ว่า คน ทำงาน ใน สิงคโปร์ มี ความเครียด จาก งาน มากขึ้น เป็น 63 % จาก เดิม ที่ เคย วัด 58 % ใน เดือน มกราคม ซึ่ง เป็นช่วง ที่ โรค โค วิด - 19 ยัง ระบาด ไม่ รุนแรง และ ยัง ไม่ มี มาตรการ จาก รัฐบาล ให้ ประชาชน ทำงาน ที่ บ้าน ( work from home ) ขณะที่ กลุ่มตัวอย่าง ในประเทศ ไทย พบ มี ความเครียด จาก งาน อยู่ ที่ 52 % เพิ่มขึ้น จาก 48 % เช่นเดียวกับ ฮ่องกง ที่ เพิ่ม จาก 46 % เป็น 47 % งานวิจัย ชิ้น นี้ ชี้ ว่า ปัจจัย หนึ่ง ที่ ทำให้ คนไทย สิงคโปร์ และ ฮ่องกง มี ความเครียด เมื่อ ต้อง ทำงาน ที่ บ้าน มากขึ้น น่าจะ มาจาก วัฒนธรรม การทำงาน ที่ ต้อง ตื่นตัว อยู่ ตลอดเวลา เพราะ กลัว จะ พลาด การ ติดต่อ งาน ที่ อาจ เข้ามา ได้ ตลอดเวลา ผ่าน ทาง โทรศัพท์ อีเมล หรือ ข้อความ ซึ่ง สอดคล้อง กับ ข้อมูล จาก กลุ่มตัวอย่าง ใน ไทย สิงคโปร์ และ ฮ่องกง ที่ ระบุ

In [11]:
print('Cross-language Output Summary: ')
with open("/content/NCLS-Corpora/code/beaver-2task+/sample_data/output_cls.txt") as f:
    for line in f:
        print(line)

Cross-language Output Summary: 
the results of a survey of thai people working in singapore, hong kong and the united states, found that most of them are stressed from their work.



## Evaluate the Model with ROUGE

In [12]:
!rouge -f '/content/NCLS-Corpora/code/beaver-2task+/sample_data/output_cls.txt' '/content/NCLS-Corpora/code/beaver-2task+/sample_data/cls-test-ref.txt'

[
  {
    "rouge-1": {
      "f": 0.354838704760666,
      "p": 0.4074074074074074,
      "r": 0.3142857142857143
    },
    "rouge-2": {
      "f": 0.033333328422222946,
      "p": 0.038461538461538464,
      "r": 0.029411764705882353
    },
    "rouge-l": {
      "f": 0.2142857093877552,
      "p": 0.25,
      "r": 0.1875
    }
  }
]


## Evaluate the Model with BertScore

In [13]:
import logging
import transformers
transformers.tokenization_utils.logger.setLevel(logging.ERROR)
transformers.configuration_utils.logger.setLevel(logging.ERROR)
transformers.modeling_utils.logger.setLevel(logging.ERROR)
%matplotlib inline

In [14]:
with open("/content/NCLS-Corpora/code/beaver-2task+/sample_data/output_cls.txt") as f:
    cands = [line.strip() for line in f]

with open("/content/NCLS-Corpora/code/beaver-2task+/sample_data/cls-test-ref.txt") as f:
    refs = [line.strip() for line in f]

In [15]:
P, R, F1 = score(cands, refs, lang='en', verbose=True)


calculating scores...
computing bert embedding.


/usr/local/lib/python3.6/dist-packages/transformers/tokenization_utils_base.py:1374: FutureWarning: The `max_len` attribute has been deprecated and will be removed in a future version, use `model_max_length` instead.
  FutureWarning,



computing greedy matching.



done in 1.10 seconds, 0.91 sentences/sec


In [16]:
print(f"System level F1 score: {F1.mean():.3f}")
print(f"System level P score: {P.mean():.3f}")
print(f"System level R score: {R.mean():.3f}")

System level F1 score: 0.891
System level P score: 0.910
System level R score: 0.873
